# Расчет проводимости трубопроводов с помощью МУК

In [1]:
import math
import numpy as np

## Прямоугольное сечение

### Угловые коэффициенты

In [2]:
def scalar_prod(v_1, v_2):
    v_1 , v_2 = np.array(v_1), np.array(v_2)
    return (v_1*v_2).sum()

def module(v):
    return math.sqrt( scalar_prod(v, v) )

def elementary(center_i : list, center_j : list, normal_i : list, normal_j : list, F_j):
    center_i, center_j = np.array(center_i), np.array(center_j)
    r = module(center_i - center_j)
    
    cos_1 = abs( scalar_prod( normal_i, center_j - center_i ) ) / r
    cos_2 = abs( scalar_prod( normal_j, center_i - center_j ) ) / r
    return cos_1*cos_2/(math.pi*r**2)*F_j

In [3]:
c_i = np.array([1/2,0,1/2])
c_j = np.array([1/2,1,1/2])

n_i = np.array([1,0,0])
n_j = np.array([-1,0,0])

In [4]:
elementary(c_i, c_j, n_i, n_j, 0.1)

0.0

In [5]:
def local(center_i, centers_2, normal_i, normal_j, F_j):
    #F_j - площадь одной ячейки коллектора
    return sum([ elementary(center_i, center, normal_i, normal_j, F_j) for center in centers_2])

def total(centers_1, centers_2,normal_1, normal_2, F_i, F_j):
    #F_i - площадь одной ячейки эмиттера
    return sum( [ local(center, centers_2, normal_1, normal_2, F_j) for center in centers_1 ] ) * F_i

### Проводимость

In [6]:
#функции разбиения каждой отдельной поверхности на ячейки
def breaking_1(a, b, l, s = 0):
    res = []
    for i in range(l):
        x = a/(2*l) + a/l*i
        for j in range(l):
            z = b/(2*l) + b/l*j
            center = [x,0,z]
            res.append(center)
    return res

def breaking_2(a, b, l, s):
    res = []
    for i in range(l):
        x = a/(2*l) + a/l*i
        for j in range(l):
            z = b/(2*l) + b/l*j
            center = [x,s,z]
            res.append(center)
    return res

def breaking_3(a, b, l, s):
    res = []
    for i in range(l):
        x = a/(2*l) + a/l*i
        for j in range(l):
            y = s/(2*l) + s/l*j
            center = [x,y,b]
            res.append(center)
    return res

def breaking_4(a, b, l, s):
    res = []
    for i in range(l):
        y = s/(2*l) + s/l*i
        for j in range(l):
            z = b/(2*l) + b/l*j
            center = [0,y,z]
            res.append(center)
    return res

def breaking_5(a, b, l, s):
    res = []
    for i in range(l):
        x = a/(2*l) + a/l*i
        for j in range(l):
            y = s/(2*l) + s/l*j
            center = [x,y,0]
            res.append(center)
    return res

def breaking_6(a, b, l, s):
    res = []
    for i in range(l):
        y = s/(2*l) + s/l*i
        for j in range(l):
            z = b/(2*l) + b/l*j
            center = [a,y,z]
            res.append(center)
    return res

In [7]:
#площади "уникальных" поверхностей - area_1 == area_2 etc
def area_1(a,b,s):
    return a*b

def area_3(a,b,s):
    return a*s

def area_4(a,b,s):
    return s*b

In [8]:
class Conduct:
    def __init__(self, a, b, L):
        self.a = a
        self.b = b
        self.L = L
        
        self.normals  = [ [0,1,0], [0,1,0], [0,0,1], [1,0,0], [0,0,1], [1,0,0] ]
        self.breaking = [ breaking_1, breaking_2, breaking_3, breaking_4, breaking_5, breaking_6 ]
        self.areas    = [ area_1, area_1, area_3, area_4, area_3, area_4 ]
        
    def calc(self, j, l_1 = 10, l_2 = 10):
        #j - номер коллектора от 1 до 6
        #l_1, l_2 - на сколько частей разбить эмиттеры и коллектор
        j-=1 #номер коллектора в массиве
        
        #параметры коллектора
        collector = self.breaking[j](self.a,self.b,l_2,self.L)
        normal_j  = self.normals[j]
        area_j    = self.areas[j](self.a,self.b,self.L)/l_2**2
        
        #параметры эмиттеров
        normals_i = [self.normals[i]   for i in range(6) if i != j]
        emitters  = [self.breaking[i]  for i in range(6) if i != j]
        areas_i   = [self.areas[i](self.a,self.b,self.L) for i in range(6) if i != j]
        
        res = 0
        for i in range(5):
            emitter  = emitters[i](self.a,self.b,l_1,self.L)
            normal_i = normals_i[i]
            area_i   = areas_i[i]/l_1**2
            
            res += total(emitter, collector, normal_i, normal_j, area_i, area_j)
            res /= areas_i[i]
        return res

### Тестирование

In [9]:
def test_1(c):
    #совпадают ли потоки на симметричные поверхности
    print(round(c.calc(2,16,16),5), round(c.calc(1, 16, 16), 5))
    print(round(c.calc(3,16,16),5), round(c.calc(5, 16, 16), 5))
    print(round(c.calc(4,16,16),5), round(c.calc(6, 16, 16), 5))

In [10]:
c = Conduct(1,1,1)
test_1(c)

1.03386 1.03386
1.03386 1.03386
1.03386 1.03386


In [ ]:
c = Conduct(1,5,1)
test_1(c)

0.52726 0.52726
